# Clasificación de Pacientes

### Librerías y cargado de datos

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [85]:
SEED = 42

In [86]:
DATASET_PATH = os.path.join('..', 'datasets', 'subset_clasif.csv')
print(DATASET_PATH)

..\datasets\subset_clasif.csv


In [87]:
df = pd.read_csv(DATASET_PATH, encoding='latin', index_col=0, dtype='uint8')
df.head(3)

C:\Users\Sebastián\AppData\Roaming\Python\Python38\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,SEXO,PAC_HOSPITALIZADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,DIAS_SINTOMAS,FALLECIDO
0,0,0,0,41,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,66,0,1,0,0,0,1,0,1,0,0,3,1
8,0,0,0,47,0,1,0,0,0,0,0,0,0,0,2,0


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 3667730 entries, 0 to 11
Data columns (total 16 columns):
 #   Column             Dtype
---  ------             -----
 0   SEXO               uint8
 1   PAC_HOSPITALIZADO  uint8
 2   NEUMONIA           uint8
 3   EDAD               uint8
 4   EMBARAZO           uint8
 5   DIABETES           uint8
 6   EPOC               uint8
 7   ASMA               uint8
 8   INMUSUPR           uint8
 9   HIPERTENSION       uint8
 10  CARDIOVASCULAR     uint8
 11  OBESIDAD           uint8
 12  RENAL_CRONICA      uint8
 13  TABAQUISMO         uint8
 14  DIAS_SINTOMAS      uint8
 15  FALLECIDO          uint8
dtypes: uint8(16)
memory usage: 83.9 MB


In [89]:
df_copy = df.copy()

## Preprocesamiento

### Tipado

In [121]:
y = df['FALLECIDO'].values.astype('float32')
X = df.drop(['FALLECIDO'], axis=1).values.astype('float32')

### Separación de Datos

In [122]:
from sklearn.model_selection import train_test_split

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=SEED)

In [131]:
[y_train.count(x) for x in np.unique(y_train)]

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [105]:
# Número de registros en cada set
X_train.shape[0], X_test.shape[0]

(2934184, 733546)

### Escalado de Datos

In [106]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [107]:
df.columns

Index(['SEXO', 'PAC_HOSPITALIZADO', 'NEUMONIA', 'EDAD', 'EMBARAZO', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR',
       'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', 'DIAS_SINTOMAS',
       'FALLECIDO'],
      dtype='object')

Rangos de las únicas variables discretas no binarias

In [108]:
# Edad
X_train[:,3].min(), X_train[:,3].max()

(0.0, 121.0)

In [109]:
# Dias Sintomas
X_train[:,-1].min(), X_train[:,-1].max()

(0.0, 174.0)

Vamos a escalar estas variables usando normalización, en parte porque no presentan una distribución normal y también porque de esa manera mantenemos similares el rango de todas las entradas de la red.

In [111]:
scaler = MinMaxScaler().fit(X_train[:,[3,-1]])
X_train[:,[3,-1]] = scaler.transform(X_train[:,[3,-1]])

Escalamos los datos de prueba usando los parámetros aprendidos en el subset de entrenamiento y el el dataset completo, ya que eso generaría contaminación de datos.

In [116]:
X_test[:,[3,-1]] = scaler.transform(X_test[:,[3,-1]])

## Modelado

In [119]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
model = keras.Sequential([
    keras.Dense(15),
    keras.Dense(10),
    keras.Dense(1)
])

In [ ]:
model.compile(optimizer='Adam', lr=0.05, loss='binary', metrics=)